# Project-2: Solving the Poisson-Boltzman Equation 

What is our goal?

Our goal is to solve the Poisson-Boltzman equation numerically. 

We expect to go throught the following stages:

1. spatial discretization of the linear diffusion operator using second order central finite differences on uniform meshes in 1D and 2D subject to various boundary conditions; 
2. spatial discretization of the non-linear reaction terms by collocation;
3. linearization by computating exact Jacobians (or otherwise);
4. root finding using Newton-type method with accuracy imposed by tolerances (or use-time integration to resolve the non-linearity);

Domenico expects that the introduction of automatic differentiation to compute the Jacobian is likely to simplify the overall solution procedure and unlock new simulation possibilities. 

## Import Libraries

In [1]:
import numpy as np
print("Succesfully imported %s -- Version: %s"%(np.__name__,np.__version__))
import scipy
print("Succesfully imported %s -- Version: %s"%(scipy.__name__,scipy.__version__))
import matplotlib.pyplot as plt
print("Succesfully imported %s"%plt.__name__)
import pandas as pd
print("Succesfully imported %s -- Version: %s"%(pd.__name__,pd.__version__))
import sympy as sym 
print("Succesfully imported %s -- Version: %s"%(sym.__name__,sym.__version__))
from scipy import optimize
print("Succesfully imported %s"%optimize.__name__)
from scipy.optimize import fsolve
print("Succesfully imported %s"%fsolve.__name__)
from scipy.signal import find_peaks
print("Succesfully imported %s"%find_peaks.__name__)
from scipy.linalg import orth
print("Succesfully imported %s"%orth.__name__)
from scipy.integrate import odeint
print("Succesfully imported %s"%odeint.__name__)

Succesfully imported numpy -- Version: 1.20.1
Succesfully imported scipy -- Version: 1.6.2
Succesfully imported matplotlib.pyplot
Succesfully imported pandas -- Version: 1.2.4
Succesfully imported sympy -- Version: 1.8
Succesfully imported scipy.optimize
Succesfully imported fsolve
Succesfully imported find_peaks
Succesfully imported orth
Succesfully imported odeint


## Section 1/: Introduction and Model Description 

Describe the Poisson-Boltzmann equation and reference solutions. For simplified cases, analytical solutions seem to be available (Ref 15 and Ref 16 on the wiki page [wiki on the Poisson-Boltzmann](https://en.wikipedia.org/wiki/Poisson–Boltzmann_equation)). These analytical solutions provide a reference for the numerical computations. Possibly the analytical solutions can be generated using symbolic computations using the sympy.dsolve() function with the bcs option to specify the boundary condtions. 

## Section 2/:  Linear Model in 1D 

Model can be linearized by evaluating the non-linear terms in a assumed previously computed solution. 

Three solution approaches are available. 

First approach: symbolically using the sympy.dsolve() function with the bcs option to specify the boundary condtions; 

Second approach: numerically using the scipy.integrate.solve_bvp() following the examples on [examples on sol ve_bvp for boundary value problems](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_bvp.html)

Third approach: add diagonal term to Poisson-1D matrix, perform linear system solve and plot solution. Copy-paste 1D-Poisson matrix here from Python intro ANM-2021. 

In [ ]:
N = 4; h=1/N; h2=h*h; xvec = np.linspace(0,1,N+1);
e = np.ones(N+1); 
A = np.diag(-e[:-1],k=1)+np.diag(2*e)+np.diag(-e[:-1],k=-1); # tridiagonal matrix 
A = (1/h2)*A; 
A[0][0]=1; A[0][1]=0;     # handling left-most boundary condition 
A[-1][-1]=2/h2; A[-1][-2]=-2/h2; # handling right-most Neumann boundary condition 
#A[-1][-1]=1; A[-1][-2]=0; # handling right-most Dirichlet boundary condition
print(A); #print(xvec)

## Section 3/: Non-Linear Model in 1D

Extend above to non-linear using exact Jacobian and Newton iteration or time-stepping.  

Three solution approaches are available. 

First approach: symbolically using the sympy.dsolve() function with the bcs option to specify the boundary condtions (as above); 

Second approach: numerically using the scipy.integrate.solve_bvp() following the examples on [examples on sol ve_bvp for boundary value problems](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_bvp.html) (as above) 

Third approach: add non-linear term to Poisson-1D matrix, perform non-linear system solve.

## Section 4/: Linear Model in 2D 

Add diagonal term to Poisson-1D matrix, perform linear system solve and plot solution. Copy-paste 2D-Poisson matrix here from Python intro ANM-2021.

##  Section 5/: Non-Linear Model in 2D 

## References

References

1. [wiki on the Poisson-Boltzmann](https://en.wikipedia.org/wiki/Poisson–Boltzmann_equation)